# PUNCH4NFDI TA5 Workshop (Dresden, 2024) ![PUNCH](punch.png "PUNCH")

## Introduction
This tutorial guides you through the process of converting a QKeras machine learning model into Vitis HLS code. You'll learn how to apply quantization-aware training (QAT) and model pruning using QKeras to optimize your model. Additionally, the tutorial covers the steps to transform the optimized model into Vitis HLS code using the HLS4ML framework, with a practical example using the MNIST dataset. You can also find the official hls4ml tutorial https://github.com/fastmachinelearning/hls4ml-tutorial.

## Requirements
- tensorflow/qkeras (build the ML model)
- HLS4ML (convert the QKeras model to HLS) https://github.com/fastmachinelearning/hls4ml
- Vitis_HLS 2022.2 (complie the HLS code and export the IP)
- Vitis/Vivado 2022.2 (generate the xclbin file that can run on Alveo card)
- Xilinx Runtime (XRT is a low level communication layer (APIs and drivers) between the host and the card.) https://xilinx.github.io/XRT/2022.2/html/index.html
- Development Target Platform (The deployment target platform is the communication layer physically implemented and flashed into the card.) https://www.xilinx.com/support/download/index.html/content/xilinx/en/downloadNav/alveo/u55c.html
- pynq >3.0.1 (a Jupyter-based framework with Python APIs for using AMD Xilinx Adaptive Computing platforms) https://pynq.readthedocs.io/en/latest/

In [ ]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
import qkeras
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os

#disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Vitis HLS path
os.environ['PATH'] = '/home/ypmen/Data/Xilinx/Vitis_HLS/2022.2/bin:' + os.environ['PATH']

## MNIST database
The MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems. The database is also widely used for training and testing in the field of machine learning. (from Wiki)

![MNIST](mnist.png "MNIST")

In [ ]:
# Load the MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images = (train_images) / 255.0
test_images = (test_images) / 255.0

train_images = train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1)
test_images = test_images.reshape(test_images.shape[0], test_images.shape[1], test_images.shape[2], 1)

## QKeras model
QKeras is a quantization extension to Keras that provides drop-in replacement for some of the Keras layers, especially the ones that creates parameters and activation layers, and perform arithmetic operations, so that we can quickly create a deep quantized version of Keras network. (https://github.com/google/qkeras)

![CNN](CNN.jpg "CNN")

The model is a simple CNN with two convolutional layers. To ensure consistency with the HLS model, both the input and output of each layer are quantized.

In [ ]:
from tensorflow.keras.layers import *
from qkeras import *
# Define the model architecture
model = keras.Sequential([
	QActivation(activation=quantized_bits(8, 2), input_shape=(28, 28, 1)),
    QConv2D(8, 3, activation=quantized_bits(8, 2), kernel_quantizer=quantized_bits(8,2,alpha=1), bias_quantizer=quantized_bits(8,2,alpha=1), kernel_initializer='lecun_uniform'),
	QActivation(activation=quantized_relu(8, 2)),
    QBatchNormalization(beta_quantizer=quantized_bits(32, 8), gamma_quantizer=quantized_bits(32, 8), mean_quantizer=quantized_bits(32, 8), variance_quantizer=quantized_bits(32, 8)),
    QActivation(activation=quantized_bits(8, 2)),
	MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
    QConv2D(8, 3, activation=quantized_bits(8, 2), kernel_quantizer=quantized_bits(8,2,alpha=1), bias_quantizer=quantized_bits(8,2,alpha=1), kernel_initializer='lecun_uniform'),
	QActivation(activation=quantized_relu(8, 2)),
    QBatchNormalization(beta_quantizer=quantized_bits(32, 8), gamma_quantizer=quantized_bits(32, 8), mean_quantizer=quantized_bits(32, 8), variance_quantizer=quantized_bits(32, 8)),
    QActivation(activation=quantized_bits(8, 2)),
	MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
	Flatten(),
   	QDense(10, kernel_quantizer=quantized_bits(8,2,alpha=1), bias_quantizer=quantized_bits(8,2,alpha=1), kernel_initializer='lecun_uniform'),
    Softmax()
])

model.summary()

In [ ]:
# Compile the model

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# callbacks
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint = ModelCheckpoint('chechpoint/test.h5', monitor='val_loss', save_best_only=True)

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=[tensorboard_callback, checkpoint])

In [ ]:
# Evaluate the model

from sklearn.metrics import classification_report

pred = model.predict(test_images)
predicted = np.argmax(pred, axis=1)
report = classification_report(test_labels, predicted)

print(report)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
# Save the model
model.save('mnist.h5')

# Load the model
# model = qkeras.utils.load_qmodel('mnist.h5')

### Pruning model

Small weights in the model are set to zero, allowing their corresponding multiplications to be removed during the synthesis of HLS code to HDL. This can reduce the usage of the fpga resources.

In [ ]:
import tensorflow_model_optimization as tfmot

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, 0),
    'block_size': (1, 1),
    'block_pooling_type': 'AVG'
}

def apply_pruning(layer):
  if isinstance(layer, QDense):
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  elif isinstance(layer, QConv2D):
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  return layer

model_for_pruning = tf.keras.models.clone_model(
    model,
    clone_function=apply_pruning,
)

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    # Log sparsity and other metrics in Tensorboard.
    tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
]

model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

In [ ]:
# Train the model
model_for_pruning.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=callbacks)

In [ ]:
# Evaluate the model

from sklearn.metrics import classification_report

pred = model.predict(test_images)
predicted = np.argmax(pred, axis=1)
report = classification_report(test_labels, predicted)

print(report)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

## HLS4ML

![hls4ml](hls4ML.jpg "hls4ml")

The QKeras model is converted to HLS using the HLS4ML framework. Configuration settings, such as precision and reuse factor, are specified in a YAML file.

In [ ]:
import hls4ml

import yaml

with open("config.yaml", 'r') as ymlfile:
	config = yaml.safe_load(ymlfile)

print("-----------------------------------")
print("Configuration")
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='model_1/hls4ml_prj',
                                                       part='xcu55c-fsvh2892-2L-e',
                                                       io_type='io_stream',
                                                       backend='Vitis')

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
# complile the QKeras model to a vitis hls project
hls_model.compile()

In [ ]:
#compare the accuracy between the qkeras and hls

labels_keras = np.argmax(model.predict(test_images), 1)
labels_hls = np.argmax(hls_model.predict(test_images), 1)

from sklearn.metrics import accuracy_score

print("Keras  Accuracy: {}".format(accuracy_score(test_labels, labels_keras)))
print("hls4ml Accuracy: {}".format(accuracy_score(test_labels, labels_hls)))


In [ ]:
# systhesis the hls code to hdl with Vitis HLS
# add "#pragma HLS INTERFACE mode=ap_ctrl_none port=return" before build

hls_model.build(csim=False, synth=True, export=True)

In [ ]:
# read the Vitis HLS synthesis report
hls4ml.report.read_vivado_report('model_1/hls4ml_prj')

## Vitis

! cd vitis/prj
! make all TARGET=hw_emu PLATFORM=xilinx_u55c_gen3x16_xdma_3_202210_1
! XCL_EMULATION_MODE=hw_emu ./host -x build_dir.hw_emu.xilinx_u55c_gen3x16_xdma_3_202210_1/krnl_ai.xclbin
! make all TARGET=hw PLATFORM=xilinx_u55c_gen3x16_xdma_3_202210_1

## Deploy on the Alveo U55C card with PYNQ

import pynq
import numpy as np
from time import sleep

In [ ]:
ol = pynq.Overlay('vitis/prj/build_dir.hw.xilinx_u55c_gen3x16_xdma_3_202210_1/krnl_ai.xclbin')

krnl_mm2s = ol.krnl_mm2s_1
krnl_s2mm = ol.krnl_s2mm_1

mm2s_in = pynq.allocate((28*28,), dtype='int8', target=ol.HBM0)
s2mm_out = pynq.allocate((10,), dtype='int16', target=ol.HBM1)

In [ ]:
mm2s_in[:] = np.fromfile('/homes/ypmen/alveo/punch/python/input_2.dat', dtype='int8')
mm2s_in.sync_to_device()
sleep(3)
krnl_mm2s.start(mm2s_in, 28*28)
krnl_s2mm.start(s2mm_out, 1)
s2mm_out.sync_from_device()
sleep(3)
print(s2mm_out/1024)